In [1]:
# Gerar áreas proporcionais dos setores censitários de acordo com suas interseções
# com os hexágonos h3 - resolução 09

# carregar bibliotecas
library('tidyverse')
library('tidylog')
library('sf')
library('mapview')

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘tidylog’


The following objects are masked from ‘package:dplyr’:

    add_count, add_tally, anti_join, count, distinct, distinct_all,
    distinct_at, distinct_if, filter, filter_all, filter_at, filter_if,
    full_join, group_by, group_by_all, group_by_at, group_by_if,
    inner_join, left_join, mutate, mutate_all, mutate_at, mutate_if,
    relocate, rename, rename_all, rename_at, rename_if, rename_with,
    right_join, sample_frac, sample_n, select, select_all, select_a

In [2]:
# Estrutura de pastas e arquivos
pasta_dados       <- "../../yellow_dados"
dados_originais   <- sprintf("%s/00_dados_originais", pasta_dados)
pasta_censo       <- sprintf("%s/CENSO", dados_originais)
pasta_ipea        <- sprintf("%s/IPEA", dados_originais)
pasta_aop_optimum <- sprintf("%s/13_aop_optimum", pasta_dados)

In [3]:
# ------------------------------------------------------------------------------
# Setore censitários - calcular área por setor
# ------------------------------------------------------------------------------

# Código do município de SP no Censo
cod_muni <- '3550308'

# Abrir shapefile de setores censitários
setores <- sprintf('%s/setores_censitarios_SP.shp', pasta_censo)

# Filtrar somente setores para a cidade de interesse
setores <- read_sf(setores) %>% filter(CD_GEOCODM == cod_muni) %>% select(cod_setor = CD_GEOCODI)

# Garantir que shapefile não apresente erros
setores <- setores %>% st_make_valid()

# Recalcular área do setor, a partir da projeção utilizada
setores <- setores %>% mutate(area_orig_setor_m2 = unclass(st_area(.)), .before = 'geometry')
# mapview(setores)
head(setores)

filter: removed 49,343 rows (72%), 18,953 rows remaining

select: renamed one variable (cod_setor) and dropped 13 variables

mutate: new variable 'area_orig_setor_m2' (double) with 18,953 unique values and 0% NA

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.9.1-CAPI-1.14.2
and GEOS at installation 3.9.1dev-CAPI-1.14.1differ”
Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



cod_setor,area_orig_setor_m2,geometry
<chr>,<dbl>,<MULTIPOLYGON [°]>
355030804000079,35862.53,MULTIPOLYGON (((-46.51778 -...
355030804000080,36314.00,MULTIPOLYGON (((-46.51888 -...
355030804000081,22274.33,MULTIPOLYGON (((-46.52015 -...
355030804000082,36724.80,MULTIPOLYGON (((-46.5214 -2...
355030804000083,29154.34,MULTIPOLYGON (((-46.52371 -...
355030804000084,63085.11,MULTIPOLYGON (((-46.5276 -2...


In [4]:
# ------------------------------------------------------------------------------
# Distribuir dados do Censo por hexágono do IPEA
# ------------------------------------------------------------------------------

# IPEA - Shape de grid hexagonal (resolução 9)
hexagonos <- sprintf('%s/aop_hex_grid_v2.gpkg', pasta_ipea)
hexagonos <- read_sf(hexagonos) %>% filter(abbrev_muni == 'spo') %>% select(id_hex)
# mapview(hexagonos)

# Caso projeções entre os dados do Censo e da intervenção sejam diferentes,
# reprojetar a intervenção para o CRS do Censo
if (st_crs(hexagonos) != st_crs(setores)) {
  hexagonos <- hexagonos %>% st_transform(crs = st_crs(setores))
}
head(hexagonos)

filter: removed 374,971 rows (96%), 15,059 rows remaining

select: dropped 3 variables (abbrev_muni, name_muni, code_muni)



id_hex,geom
<chr>,<POLYGON [°]>
89a81009a8bffff,POLYGON ((-46.43596 -23.586...
89a8108dca7ffff,POLYGON ((-46.77886 -23.897...
89a81015a8bffff,"POLYGON ((-46.74 -23.7229, ..."
89a810019d7ffff,POLYGON ((-46.62778 -23.654...
89a8100d9d7ffff,POLYGON ((-46.62335 -23.504...
89a81039a8bffff,POLYGON ((-46.73109 -23.422...


In [5]:
# Recortar setores dentro da area de entorno da intervenção
setores_hex <- st_intersection(setores, hexagonos)

although coordinates are longitude/latitude, st_intersection assumes that they are planar

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”


In [6]:
# Calcular área proporcional à interseção com o hexágonos
setores_hex <- setores_hex %>% mutate(area_setor_hex_m2 = as.numeric(st_area(.)), 
                                      area_prop = area_setor_hex_m2 / area_orig_setor_m2,
                                      .before = 'geometry')

head(setores_hex)

mutate: new variable 'area_setor_hex_m2' (double) with 70,349 unique values and 0% NA

        new variable 'area_prop' (double) with 68,077 unique values and 0% NA



cod_setor,area_orig_setor_m2,id_hex,area_setor_hex_m2,area_prop,geometry
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<POLYGON [°]>
355030847000190,67587.63,89a81009a8bffff,39997.43,0.591786208,POLYGON ((-46.43809 -23.586...
355030847000192,451779.55,89a81009a8bffff,66156.94,0.146436335,POLYGON ((-46.43647 -23.586...
355030852000051,2103500.76,89a8108dca7ffff,64531.09,0.030677948,POLYGON ((-46.77808 -23.895...
355030852000053,2977044.04,89a8108dca7ffff,40883.14,0.013732797,POLYGON ((-46.78082 -23.894...
355030843000097,6040579.31,89a81015a8bffff,25944.45,0.004295027,POLYGON ((-46.74133 -23.722...
355030846000242,1901910.53,89a81015a8bffff,79820.29,0.041968477,POLYGON ((-46.74133 -23.722...


In [8]:
# Ordenar para exportar
setores_hex <- setores_hex %>% arrange(cod_setor, id_hex)
head(setores_hex, 15)

cod_setor,area_orig_setor_m2,id_hex,area_setor_hex_m2,area_prop,geometry
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<POLYGON [°]>
355030801000001,72144.99,89a8100d423ffff,40023.1080,0.554759346,POLYGON ((-46.57234 -23.567...
355030801000001,72144.99,89a8100d42bffff,186.5239,0.002585403,POLYGON ((-46.56955 -23.565...
355030801000001,72144.99,89a8100d433ffff,20834.7340,0.288789751,POLYGON ((-46.57005 -23.569...
355030801000001,72144.99,89a8100d437ffff,4793.5848,0.066443764,POLYGON ((-46.5724 -23.5683...
355030801000001,72144.99,89a8100d43bffff,6307.0426,0.087421767,POLYGON ((-46.56956 -23.569...
355030801000002,71635.91,89a8100d423ffff,613.4269,0.008563120,POLYGON ((-46.56936 -23.566...
355030801000002,71635.91,89a8100d42bffff,42916.2756,0.599088887,POLYGON ((-46.56938 -23.566...
355030801000002,71635.91,89a8100d43bffff,28106.2024,0.392347968,POLYGON ((-46.56878 -23.568...
355030801000003,55977.67,89a8100d407ffff,9165.5267,0.163735420,POLYGON ((-46.5676 -23.5703...


In [9]:
# Gravar resultados
out_file <- sprintf('%s/hex_grid_sp_res09_areas_setores_censitarios.gpkg', pasta_aop_optimum)
st_write(setores_hex, out_file, driver = 'GPKG', append = FALSE)

Deleting layer `hex_grid_sp_res09_areas_setores_censitarios' using driver `GPKG'
Writing layer `hex_grid_sp_res09_areas_setores_censitarios' to data source `../../yellow_dados/13_aop_optimum/hex_grid_sp_res09_areas_setores_censitarios.gpkg' using driver `GPKG'
Writing 70349 features with 5 fields and geometry type Unknown (any).
